# Lorenz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [ ]:
def prepare_dataset(rho):
  nn_input = np.zeros((number_of_datasets*(len(t)-1),3))
  nn_output = np.zeros_like(nn_input)

  def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# initial conditions
  #x0 = -15 + 30 * 0.6 * np.ones((number_of_datasets, 3)) 
  x0 = np.random.random((number_of_datasets, 3))

  x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t)
                  for x0_j in x0])
  
  for j in range(number_of_datasets):
    nn_input[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,:-1,:]
    nn_output[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,1:,:]
    x, y, z = x_t[j,:,:].T


  return nn_input, nn_output


<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Dataset generation

In [ ]:
## Simulate the Lorenz System rho = 10, 28, 35
np.random.seed(123)
dt = 0.001
T = 10
t = np.arange(0, T+dt, dt)
beta = 8/3
sigma = 10
number_of_datasets = 10

rho = 10
nn_in_10, nn_out_10 = prepare_dataset(rho)
nn_in_10 = nn_in_10 
nn_in_10 = np.concatenate((nn_in_10,np.ones((len(nn_in_10),1))*rho),axis=1)
nn_out_10 = nn_out_10 

rho = 28
nn_in_28, nn_out_28 = prepare_dataset(rho)
nn_in_28 = nn_in_28 
nn_in_28 = np.concatenate((nn_in_28,np.ones((len(nn_in_28),1))*rho),axis=1)
nn_out_28 = nn_out_28 

rho = 35
nn_in_35, nn_out_35 = prepare_dataset(rho)
nn_in_35 = nn_in_35 
nn_in_35 = np.concatenate((nn_in_35,np.ones((len(nn_in_35),1))*rho),axis=1)
nn_out_35 = nn_out_35 


nn_input = np.concatenate((nn_in_10,nn_in_28,nn_in_35),axis=0)
nn_output = np.concatenate((nn_out_10, nn_out_28, nn_out_35), axis = 0)

<br>
<br>
<br>
<br>
<br>
<br>

# Train the Neural Network 

In [ ]:
deep_approx = tf.keras.models.Sequential()
deep_approx.add(tf.keras.layers.Dense(10, input_dim=4, activation='relu'))
deep_approx.add(tf.keras.layers.Dense(10, activation='relu'))
deep_approx.add(tf.keras.layers.Dense(3, activation='linear'))

decayRate = 1e-4
nrSamplesPostValid = 2
learningRate = 1e-3

nEpochs = 1000
batchSize = 128
verbosity = 1

adam = tf.keras.optimizers.Adam( learning_rate = learningRate, decay = decayRate )
# Compile model
deep_approx.compile(loss='mse', optimizer=adam)

# Fit!
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
History = deep_approx.fit(nn_input, nn_output, epochs=nEpochs )


<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Test the Neural Network prediction on new dataset with different rho value 

### Solving for rho = 17 

In [ ]:
rho = 17
np.random.seed(139)
num_traj = 1

nn_flow = np.zeros((num_traj, len(t), 4))
nn_flow[:, 0, :-1] = -15 + 30 * 0.6 * np.ones((num_traj, 3))
#nn_flow[:, 0, :-1] = np.random.random((num_traj, 3))*10

nn_flow[:,:,3] = np.ones((num_traj,len(t)))*rho
for jj, tval in enumerate(t[:-1]):
  nn_flow[:, jj+1, :-1] = deep_approx.predict(nn_flow[:, jj, :])


def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
  x, y, z = x_y_z
  return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

nn_flow_simulation = nn_flow[:,:,0:3]
x_t = np.array([integrate.odeint(lorenz_deriv, nn_flow_simulation[i, 0, :], t) for i in range(num_traj)])


nn_flow_17 = nn_flow
x_t_17 = x_t

### Solving for rho = 40

In [ ]:
rho = 40

nn_flow = np.zeros((num_traj, len(t), 4))
nn_flow[:, 0, :-1] = -15 + 30 * 0.6 * np.ones((num_traj, 3))
#nn_flow[:, 0, :-1] = np.random.random((num_traj, 3))*10

nn_flow[:,:,3] = np.ones((num_traj,len(t)))*rho
for jj, tval in enumerate(t[:-1]):
  nn_flow[:, jj+1, :-1] = deep_approx.predict(nn_flow[:, jj, :])


def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
  x, y, z = x_y_z
  return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

nn_flow_simulation = nn_flow[:,:,0:3]
x_t = np.array([integrate.odeint(lorenz_deriv, nn_flow_simulation[i, 0, :], t) for i in range(num_traj)])


nn_flow_40 = nn_flow
x_t_40 = x_t

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

x_17 = np.zeros((len(t),num_traj))
y_17 = np.zeros((len(t),num_traj))
z_17 = np.zeros((len(t),num_traj))

x_40 = np.zeros((len(t),num_traj))
y_40 = np.zeros((len(t),num_traj))
z_40 = np.zeros((len(t),num_traj))

xd_17 = np.zeros((len(t),num_traj))
yd_17 = np.zeros((len(t),num_traj))
zd_17 = np.zeros((len(t),num_traj))

xd_40 = np.zeros((len(t),num_traj))
yd_40 = np.zeros((len(t),num_traj))
zd_40 = np.zeros((len(t),num_traj))


for j in range(num_traj):
    x_17[:,j], y_17[:,j], z_17[:,j] = x_t_17[j, :, :].T
    xd_17[:,j], yd_17[:,j], zd_17[:,j] = nn_flow_17[j, :, :-1].T
    x_40[:,j], y_40[:,j], z_40[:,j] = x_t_40[j, :, :].T
    xd_40[:,j], yd_40[:,j], zd_40[:,j] = nn_flow_40[j, :, :-1].T

    ax.plot(x_17[:,j], y_17[:,j], z_17[:,j],'b' ,linewidth=1)
    ax.plot(xd_17[:,j], yd_17[:,j], zd_17[:,j], '--b', lw=1)
    ax.plot(x_40[:,j], y_40[:,j], z_40[:,j],'r' ,linewidth=1)
    ax.plot(xd_40[:,j], yd_40[:,j], zd_40[:,j], '--r', lw=1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.scatter(x_17[0], y_17[0], z_17[0], color='b')
    ax.scatter(x_40[0], y_40[0], z_40[0], color='b')
    ax.legend(['\u03C1 = 17, ODE45 ','\u03C1 = 17, NN ' ,'\u03C1 = 40, ODE45', '\u03C1 = 40, NN '], loc = 'upper right')
             
ax.view_init(18, -13)
plt.show()

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(10,10))
ax[0].set_title('Coordinate evolution over time for \u03C1 = 17 and \u03C1 = 40' )
ax[0].plot(t,x_17,'b')
ax[0].plot(t,xd_17, 'b--')
ax[0].plot(t,x_40,'r')
ax[0].plot(t,xd_40, 'r--')
ax[0].grid() 
ax[0].legend(['\u03C1 = 17, ODE45 ','\u03C1 = 17, NN ' ,'\u03C1 = 40, ODE45', '\u03C1 = 40, NN '], loc = 'upper right')
ax[0].set_ylabel('x coordinate')


ax[1].plot(t,y_17,'b')
ax[1].plot(t,yd_17, 'b--')
ax[1].plot(t,y_40,'r')
ax[1].plot(t,yd_40, 'r--')
ax[1].grid()
ax[1].set_ylabel('y coordinate')

ax[2].plot(t,z_17,'b')
ax[2].plot(t,zd_17, 'b--')
ax[2].plot(t,z_40,'r')
ax[2].plot(t,zd_40, 'r--')
ax[2].grid()
ax[2].set_xlabel('Time [s]')
ax[2].set_ylabel('z coordinate')

In [ ]:


x_mean = np.mean(x,axis=1)
y_mean = np.mean(y,axis=1)
z_mean = np.mean(z,axis=1)

xd_upper = np.percentile(xd,0.5,axis=1)
yd_upper = np.percentile(yd,0.5,axis=1)
zd_upper = np.percentile(zd,0.5,axis=1)

xd_lower = np.percentile(xd,95.5,axis=1)
yd_lower = np.percentile(yd,95.5,axis=1)
zd_lower = np.percentile(zd,95.5,axis=1)

xd_mean = np.mean(xd,axis=1)
yd_mean = np.mean(yd,axis=1)
zd_mean = np.mean(zd,axis=1)

fig, ax = plt.subplots(3,1)
ax[0].set_title('Coordinate evolution over time for rho = %d' %rho)
ax[0].plot(t,x_mean,'b')
ax[0].plot(t,xd_mean, 'r--')
ax[0].plot(t, xd_lower, 'g', alpha=0.6)
ax[0].plot(t, xd_upper, 'g', alpha=0.6)
ax[0].fill_between(t, xd_lower, xd_upper, color='g', alpha=0.3)
ax[0].grid() 
ax[0].set_ylabel('x coordinate')
ax[0].legend(['True','Predicted'])

ax[1].plot(t,y_mean,'b')
ax[1].plot(t,yd_mean, 'r--')
ax[1].plot(t, yd_lower, 'g', alpha=0.6)
ax[1].plot(t, yd_upper, 'g', alpha=0.6)
ax[1].fill_between(t, yd_lower, yd_upper, color='g', alpha=0.3)
ax[1].grid()
ax[1].set_ylabel('y coordinate')

ax[2].plot(t,z_mean,'b')
ax[2].plot(t,zd_mean, 'r--')
ax[2].plot(t, zd_lower, 'g', alpha=0.6)
ax[2].plot(t, zd_upper, 'g', alpha=0.6)
ax[2].fill_between(t, zd_lower, zd_upper, color='g', alpha=0.3)
ax[2].grid()
ax[2].set_xlabel('Time [s]')
ax[2].set_ylabel('z coordinate')